In [69]:
# !pip install \
#   langchain_community \
#   langchain_pinecone \
#   langchain_openai \
#   unstructured \
#   langchain-text-splitters \
#   pinecone-text    

In [70]:
from langchain_pinecone import PineconeVectorStore
from langchain_community.retrievers import (
    PineconeHybridSearchRetriever)
from pinecone.grpc import PineconeGRPC as Pinecone
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain_community.document_loaders import DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain_openai import OpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain.document_loaders import TextLoader
from langchain_core.runnables import Runnable
from pinecone import ServerlessSpec
import pandas as pd
import itertools
from sentence_transformers import SentenceTransformer
from tqdm.auto import tqdm
from dotenv import load_dotenv
import pinecone
import uuid
import os
import glob
import hashlib
from tqdm.autonotebook import tqdm
from langchain.storage import InMemoryStore
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

In [34]:
#  import nltk
#  nltk.download('punkt')

In [71]:
import os
from dotenv import load_dotenv

load_dotenv()
# Fetch API keys from environment variables
openai_api_key = os.getenv('OPENAI_API_KEY')
pinecone_api_key = os.getenv('PINECONE_API_KEY')


# Set the environment variables
if openai_api_key:
    os.environ['OPENAI_API_KEY'] = openai_api_key
if pinecone_api_key:
    os.environ['PINECONE_API_KEY'] = pinecone_api_key 

#Verify that the keys are loaded
#print(f"OpenAI API Key: {os.environ.get('OPENAI_API_KEY')}")
#print(f"Pinecone API Key: {os.environ.get('PINECONE_API_KEY')}")

In [72]:
directory = 'data'

def load_docs(directory):
    loader = DirectoryLoader(directory)
    docs = loader.load()
    return docs

docs = load_docs(directory)

In [37]:
#docs

In [38]:
#print(len(docs))

In [39]:
# print(docs[0].page_content[0:100])
# print(docs[0].metadata)

In [73]:

os.environ["HUGGINGFACEHUB_API_TOKEN"]="HUGGINGFACEHUB_API_TOKEN"
embeddings = OpenAIEmbeddings(
    model="text-embedding-ada-002",  #response time is 9s  #infloat/e5-base-V2 has 3.53sec response time.
)
embeddings

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x31f1b31d0>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x323d7f5c0>, model='text-embedding-ada-002', dimensions=None, deployment='text-embedding-ada-002', openai_api_version=None, openai_api_base=None, openai_api_type=None, openai_proxy=None, embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

In [74]:
import getpass
import os
import time
from pinecone import Pinecone, ServerlessSpec

if not os.getenv("PINECONE_API_KEY"):
    os.environ["PINECONE_API_KEY"] = getpass.getpass("Enter your Pinecone API key: ")

pinecone_api_key = os.environ.get("PINECONE_API_KEY")

pc = Pinecone(api_key=pinecone_api_key)
import time

index_name = "test-2"  # change if desired

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=3072,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)
#pc.list_indexes()

In [42]:
# import os
# import uuid
# from PyPDF2 import PdfReader  # For reading PDF files
# import docx
# from langchain.document_loaders import DirectoryLoader  # Assuming DirectoryLoader is part of LangChain

# directory = 'data'

# # Function to load documents from the directory
# def load_docs(directory):
#     loader = DirectoryLoader(directory)
#     docs = loader.load()  # Load documents using the DirectoryLoader
#     return docs

# # Upload documents and store metadata including source and text
# def upload_documents(docs, embeddings, index):
#     for doc in docs:
#         # Extract document content and metadata
#         document_text = doc.page_content
#         source = doc.metadata.get('source', 'unknown')  # Assuming 'source' is part of doc.metadata

#         if document_text:  # Only proceed if the document was read successfully
#             # 1. Generate embedding for the document
#             vector = embeddings.embed_documents([document_text])[0]  # Use embed_documents for document embeddings

#             # 2. Generate a unique vector ID (UUID)
#             vector_id = str(uuid.uuid4())  # Generate a unique UUID
            
#             # 3. Prepare metadata (source and content preview)
#             metadata = {
#                 "source": source,  # The document name as source
#                 "text": document_text[:100]  # Store first 100 characters as a preview of the document
#             }
            
#             # 4. Upsert the vector into Pinecone with metadata
#             index.upsert(vectors=[(vector_id, vector, metadata)])  # Make sure to include metadata here

#             # 5. Print vector ID and document name
#             print(f"Uploaded Document - Vector ID: {vector_id}, Document Source: {source}")

#     print("All documents uploaded successfully!")

# # Load documents using load_docs function
# docs = load_docs(directory)

# # Call the function to upload documents into Pinecone vector store
# upload_documents(docs, embeddings, index)


In [43]:
# import os
# import uuid
# from PyPDF2 import PdfReader  # For reading PDF files
# import docx
# from langchain.document_loaders import DirectoryLoader  # Assuming DirectoryLoader is part of LangChain

# directory = 'data'

# # Function to load documents from the directory
# def load_docs(directory):
#     loader = DirectoryLoader(directory)
#     docs = loader.load()  # Load documents using the DirectoryLoader
#     return docs

# # Upload documents and store metadata including source and text
# def upload_documents(docs, embeddings, index, tenant_id):
#     for doc in docs:
#         # Extract document content and metadata
#         document_text = doc.page_content
#         source = doc.metadata.get('source', 'unknown')  # Assuming 'source' is part of doc.metadata

#         if document_text:  # Only proceed if the document was read successfully
#             # 1. Generate embedding for the document
#             vector = embeddings.embed_documents([document_text])[0]  # Use embed_documents for document embeddings

#             # 2. Generate a unique vector ID (UUID)
#             vector_id = str(uuid.uuid4())  # Generate a unique UUID
            
#             # 3. Prepare metadata (source, tenant_id, and content preview)
#             metadata = {
#                 "source": source,  # The document name as source
#                 "tenant_id": tenant_id,  # Add tenant-specific ID
#                 "text": document_text[:100]  # Store first 100 characters as a preview of the document
#             }
            
#             # 4. Upsert the vector into Pinecone with metadata
#             index.upsert(vectors=[{"id": vector_id, "values": vector, "metadata": metadata}])  # Include metadata here

#             # 5. Print vector ID and document name
#             print(f"Uploaded Document - Vector ID: {vector_id}, Document Source: {source}, Tenant ID: {tenant_id}")

#     print("All documents uploaded successfully!")

# # Load documents using load_docs function
# docs = load_docs(directory)

# # Example of embedding generation and Pinecone index setup
# # embeddings and index should already be initialized
# tenant_id = "example_tenant_id"  # Add your tenant_id here

# # Call the function to upload documents into Pinecone vector store
# upload_documents(docs, embeddings, index, tenant_id)


In [44]:
# import os
# import uuid
# from PyPDF2 import PdfReader  # For reading PDF files
# import docx
# from langchain.document_loaders import DirectoryLoader  # Assuming DirectoryLoader is part of LangChain

# directory = 'data'

# # Function to load documents from the directory
# def load_docs(directory):
#     loader = DirectoryLoader(directory)
#     docs = loader.load()  # Load documents using the DirectoryLoader
#     return docs

# # Upload documents and store metadata including source and text
# def upload_documents(docs, embeddings, index):
#     for doc in docs:
#         # Extract document content and metadata
#         document_text = doc.page_content
#         source = doc.metadata.get('source', 'unknown')  # Assuming 'source' is part of doc.metadata

#         if document_text:  # Only proceed if the document was read successfully
#             # 1. Generate embedding for the document
#             vector = embeddings.embed_documents([document_text])[0]  # Use embed_documents for document embeddings

#             # 2. Generate a unique vector ID (UUID)
#             vector_id = str(uuid.uuid4())  # Generate a unique UUID
            
#             # 3. Prepare metadata (source and content preview)
#             metadata = {
#                 "source": source,  # The document name as source
#                 "text": document_text[:100]  # Store first 100 characters as a preview of the document
#             }
            
#             # 4. Upsert the vector into Pinecone with metadata
#             index.upsert(vectors=[{"id": vector_id, "values": vector, "metadata": metadata}])  # Include metadata here

#             # 5. Print vector ID and document name
#             print(f"Uploaded Document - Vector ID: {vector_id}, Document Source: {source}")

#     print("All documents uploaded successfully!")

# # Load documents using load_docs function
# docs = load_docs(directory)

# # Example of embedding generation and Pinecone index setup
# # embeddings and index should already be initialized

# # Call the function to upload documents into Pinecone vector store
# upload_documents(docs, embeddings, index)


In [45]:
# import os
# import uuid
# from PyPDF2 import PdfReader  # For reading PDF files
# import docx
# from langchain.document_loaders import DirectoryLoader


# directory = 'data'
# def load_docs(directory):
#     loader = DirectoryLoader(directory)
#     docs = loader.load()  # Load documents using the DirectoryLoader
#     return docs

# chunk_size = 1000  # Adjust based on your use case
# chunk_overlap = 100  # Adjust based on your use case

# # Function to read text from different file types
# def read_document(file_path):
#     if file_path.endswith('.txt'):
#         with open(file_path, 'r', encoding='utf-8') as file:
#             return file.read()
#     elif file_path.endswith('.pdf'):
#         with open(file_path, 'rb') as file:
#             reader = PdfReader(file)
#             return "\n".join(page.extract_text() for page in reader.pages if page.extract_text())
#     elif file_path.endswith('.docx'):
#         doc = docx.Document(file_path)
#         return "\n".join(paragraph.text for paragraph in doc.paragraphs)
#     else:
#         return None  # Unsupported file type

# # Function to upload document chunks with ID prefixes
# def upload_documents(directory, embeddings, index):
#     text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    
#     # Loop through files in the directory
#     for filename in os.listdir(directory):
#         file_path = os.path.join(directory, filename)
#         document_text = read_document(file_path)
        
#         if document_text:
#             # Generate the document ID prefix
#             doc_id_prefix = f"{filename.split('.')[0]}"

#             # Split the document into chunks
#             chunks = text_splitter.split_text(document_text)
            
#             vectors = []
#             for i, chunk in enumerate(chunks):
#                 # Create unique ID for each chunk
#                 chunk_id = f"{doc_id_prefix}#chunk{i+1}"
#                 # Generate embedding for each chunk
#                 vector = embeddings.embed_documents([chunk])[0]
#                 # Prepare metadata (source and content preview)
#                 metadata = {
#                     "source": filename,
#                     "page_content": chunk[:100],  # Store first 100 characters as a preview of the chunk
#                 }
#                 # Append vector with ID and metadata
#                 vectors.append({"id": chunk_id, "values": vector, "metadata": metadata})

#             # Upsert the vectors (chunks) into Pinecone with metadata
#             index.upsert(vectors=vectors, namespace="")

#             print(f"Uploaded Document - Prefix: {doc_id_prefix}, Total Chunks: {len(chunks)}")

#     print("All documents uploaded successfully!")

# # Call the function to upload documents from the directory
# upload_documents(directory, embeddings, index)

# # Function to list all chunks of a document by prefix
# def list_document_chunks(doc_id_prefix):
#     chunk_ids = []
#     for ids in index.list(prefix=f"{doc_id_prefix}#", namespace=""):
#         chunk_ids.extend(ids)
#     return chunk_ids


In [46]:
# import os
# import uuid
# from PyPDF2 import PdfReader  # For reading PDF files
# import docx
# import random
# from langchain.document_loaders import DirectoryLoader
# from langchain.text_splitter import RecursiveCharacterTextSplitter


# directory = 'data'
# def load_docs(directory):
#     loader = DirectoryLoader(directory)
#     docs = loader.load()  # Load documents using the DirectoryLoader
#     return docs

# chunk_size = 1000  # Adjust based on your use case
# chunk_overlap = 100  # Adjust based on your use case

# # Function to read text from different file types
# def read_document(file_path):
#     if file_path.endswith('.txt'):
#         with open(file_path, 'r', encoding='utf-8') as file:
#             return file.read()
#     elif file_path.endswith('.pdf'):
#         with open(file_path, 'rb') as file:
#             reader = PdfReader(file)
#             return "\n".join(page.extract_text() for page in reader.pages if page.extract_text())
#     elif file_path.endswith('.docx'):
#         doc = docx.Document(file_path)
#         return "\n".join(paragraph.text for paragraph in doc.paragraphs)
#     else:
#         return None  # Unsupported file type

# # Function to generate a random 8-digit ID
# def generate_random_id():
#     return str(random.randint(10000000, 99999999))

# # Function to upload document chunks with a random 8-digit ID prefix
# def upload_documents(directory, embeddings, index):
#     text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    
#     # Loop through files in the directory
#     for filename in os.listdir(directory):
#         file_path = os.path.join(directory, filename)
#         document_text = read_document(file_path)
        
#         if document_text:
#             # Generate a random 8-digit document ID prefix
#             doc_id_prefix = generate_random_id()

#             # Split the document into chunks
#             chunks = text_splitter.split_text(document_text)
            
#             vectors = []
#             for i, chunk in enumerate(chunks):
#                 # Create unique ID for each chunk
#                 chunk_id = f"{doc_id_prefix}#chunk{i+1}"
#                 # Generate embedding for each chunk
#                 vector = embeddings.embed_documents([chunk])[0]
#                 # Prepare metadata (source and content preview)
#                 metadata = {
#                     "source": filename,
#                     "page_content": chunk[:100],  # Store first 100 characters as a preview of the chunk
#                 }
#                 # Append vector with ID and metadata
#                 vectors.append({"id": chunk_id, "values": vector, "metadata": metadata})

#             # Upsert the vectors (chunks) into Pinecone with metadata
#             index.upsert(vectors=vectors, namespace="")

#             print(f"Uploaded Document - Prefix: {doc_id_prefix}, Total Chunks: {len(chunks)}")

#     print("All documents uploaded successfully!")

# # Function to list all chunks of a document by prefix
# def list_document_chunks(doc_id_prefix):
#     chunk_ids = []
#     for ids in index.list(prefix=f"{doc_id_prefix}#", namespace=""):
#         chunk_ids.extend(ids)
#     return chunk_ids

# # Call the function to upload documents from the directory
# upload_documents(directory, embeddings, index)


## Prefix

In [75]:
import os
import concurrent.futures
from PyPDF2 import PdfReader  # For reading PDF files
import docx
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


directory = 'data'
chunk_size = 1000  # Adjust based on your use case
chunk_overlap = 100  # Adjust based on your use case
batch_size = 50  # Number of chunks per batch for upsert to Pinecone


# Function to read text from different file types
def read_document(file_path):
    if file_path.endswith('.txt'):
        with open(file_path, 'r', encoding='utf-8') as file:
            return file.read()
    elif file_path.endswith('.pdf'):
        with open(file_path, 'rb') as file:
            reader = PdfReader(file)
            return "\n".join(page.extract_text() for page in reader.pages if page.extract_text())
    elif file_path.endswith('.docx'):
        doc = docx.Document(file_path)
        return "\n".join(paragraph.text for paragraph in doc.paragraphs)
    else:
        return None  # Unsupported file type


# Function to process and upsert a single document
def process_and_upload_document(file_path, embeddings, index, tenant_ID):
    document_text = read_document(file_path)

    if document_text:
        # Generate the document ID prefix
        doc_id_prefix = f"{os.path.basename(file_path).split('.')[0]}"
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)

        # Split the document into chunks
        chunks = text_splitter.split_text(document_text)
        vectors = []
        
        # Create vectors for each chunk
        for i, chunk in enumerate(chunks):
            chunk_id = f"{doc_id_prefix}#chunk{i+1}"
            vector = embeddings.embed_documents([chunk])[0]
            metadata = {
                "filename": os.path.basename(file_path),
                "tenant_ID": tenant_ID,  # Add tenant_ID to metadata
            }
            vectors.append({"id": chunk_id, "values": vector, "metadata": metadata})

            # Batch upsert after processing `batch_size` chunks
            if len(vectors) >= batch_size:
                index.upsert(vectors=vectors, namespace="")
                vectors.clear()  # Clear the batch after uploading

        # Upsert any remaining vectors
        if vectors:
            index.upsert(vectors=vectors, namespace="")
        
        print(f"Uploaded Document - Prefix: {doc_id_prefix}, Total Chunks: {len(chunks)}")
    else:
        print(f"Skipping unsupported or empty file: {file_path}")


# Function to upload documents in parallel
def upload_documents_concurrently(directory, embeddings, index, tenant_ID):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = []
        for filename in os.listdir(directory):
            file_path = os.path.join(directory, filename)
            futures.append(executor.submit(process_and_upload_document, file_path, embeddings, index, tenant_ID))
        
        # Wait for all threads to complete
        for future in concurrent.futures.as_completed(futures):
            future.result()

    print("All documents uploaded successfully!")


# Call the function to upload documents from the directory concurrently
tenant_ID = "your_tenant_ID_here"  # Add tenant ID here or pass dynamically
upload_documents_concurrently(directory, embeddings, index, tenant_ID)


# Function to list all chunks of a document by prefix
def list_document_chunks(doc_id_prefix):
    chunk_ids = []
    for ids in index.list(prefix=f"{doc_id_prefix}#", namespace=""):
        chunk_ids.extend(ids)
    return chunk_ids


Skipping unsupported or empty file: data/.DS_Store
Uploaded Document - Prefix: Activity Logs Module , Total Chunks: 1
Uploaded Document - Prefix: Clients - Comms Module , Total Chunks: 1
Uploaded Document - Prefix: Virtual Visits Module , Total Chunks: 2
Uploaded Document - Prefix: Clients - Cases Module , Total Chunks: 4
Uploaded Document - Prefix: Files Module , Total Chunks: 2
Uploaded Document - Prefix: Comms Module , Total Chunks: 2
Uploaded Document - Prefix: Clients - Billings Module , Total Chunks: 2
Uploaded Document - Prefix: Leads Module , Total Chunks: 4
Uploaded Document - Prefix: Clients - Clients Module , Total Chunks: 5
Uploaded Document - Prefix: Reports Module , Total Chunks: 2
Uploaded Document - Prefix:  Billings Module , Total Chunks: 6
Uploaded Document - Prefix: Cases Module , Total Chunks: 8
Uploaded Document - Prefix: Calendar Module , Total Chunks: 4
Uploaded Document - Prefix: Appointments Module , Total Chunks: 3
Uploaded Document - Prefix: Settings Module, 

In [48]:
import os
import concurrent.futures


# Function to list all chunks of a document by prefix
def list_document_chunks(doc_id_prefix):
    chunk_ids = []
    # Assuming index.list supports pagination or has a mechanism to retrieve chunks efficiently
    for ids in index.list(prefix=f"{doc_id_prefix}#", namespace=""):
        chunk_ids.extend(ids)
    return chunk_ids


# Function to delete all chunks of a document by prefix in parallel
def delete_document_by_prefix(doc_id_prefix, batch_size=100):
    chunk_ids = list_document_chunks(doc_id_prefix)
    
    if chunk_ids:
        def delete_chunks_batch(batch):
            index.delete(ids=batch, namespace="")
        
        # Batch the chunk deletions to reduce overhead
        with concurrent.futures.ThreadPoolExecutor() as executor:
            # Split chunk_ids into batches of size `batch_size`
            batches = [chunk_ids[i:i + batch_size] for i in range(0, len(chunk_ids), batch_size)]
            futures = [executor.submit(delete_chunks_batch, batch) for batch in batches]

            # Wait for all batch deletions to complete
            for future in concurrent.futures.as_completed(futures):
                future.result()

        print(f"Deleted {len(chunk_ids)} chunks for document: {doc_id_prefix}")
    else:
        print(f"No chunks found for document: {doc_id_prefix}")


# Function to upload documents in parallel (unchanged)
def upload_documents_concurrently(directory, embeddings, index, tenant_ID):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = []
        for filename in os.listdir(directory):
            file_path = os.path.join(directory, filename)
            futures.append(executor.submit(process_and_upload_document, file_path, embeddings, index, tenant_ID))
        
        # Wait for all threads to complete
        for future in concurrent.futures.as_completed(futures):
            future.result()

    print("All documents uploaded successfully!")


# Example usage
doc_id_prefix = "Settings Module"  # Replace with your actual document name

# Delete all chunks for a document in parallel
delete_document_by_prefix(doc_id_prefix)


Deleted 7 chunks for document: Settings Module


In [49]:
# import os
# import uuid
# import random
# from PyPDF2 import PdfReader  # For reading PDF files
# import docx
# from langchain.document_loaders import DirectoryLoader
# from langchain.text_splitter import RecursiveCharacterTextSplitter

# # directory = 'data'
# chunk_size = 1000  # Adjust based on your use case
# chunk_overlap = 100  # Adjust based on your use case

# # # Function to load documents
# # def load_docs(directory):
# #     loader = DirectoryLoader(directory)
# #     docs = loader.load()  # Load documents using the DirectoryLoader
# #     return docs

# # Function to read text from different file types
# def read_document(file_path):
#     if file_path.endswith('.txt'):
#         with open(file_path, 'r', encoding='utf-8') as file:
#             return file.read()
#     elif file_path.endswith('.pdf'):
#         with open(file_path, 'rb') as file:
#             reader = PdfReader(file)
#             return "\n".join(page.extract_text() for page in reader.pages if page.extract_text())
#     elif file_path.endswith('.docx'):
#         doc = docx.Document(file_path)
#         return "\n".join(paragraph.text for paragraph in doc.paragraphs)
#     else:
#         return None  # Unsupported file type

# # Function to generate a 14-digit random ID
# def generate_14_digit_random_id():
#     return str(random.randint(10000000000000, 99999999999999))

# # Function to upload document chunks with a random 14-digit ID prefix
# def upload_documents(directory, embeddings, index, tenant_ID):
#     text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    
#     # Loop through files in the directory
#     for filename in os.listdir(directory):
#         file_path = os.path.join(directory, filename)
#         document_text = read_document(file_path)
        
#         if document_text:
#             # Generate a random 14-digit document ID prefix
#             doc_id_prefix = generate_14_digit_random_id()

#             # Split the document into chunks
#             chunks = text_splitter.split_text(document_text)
            
#             vectors = []
#             for i, chunk in enumerate(chunks):
#                 # Create unique ID for each chunk
#                 chunk_id = f"{doc_id_prefix}#chunk{i+1}"
#                 # Generate embedding for each chunk
#                 vector = embeddings.embed_documents([chunk])[0]
#                 # Prepare metadata (source, tenant_ID, and content preview)
#                 metadata = {
#                     "source": filename,
#                     "tenant_ID": tenant_ID,  # Store the tenant ID as metadata
#                     "page_content": chunk[:100],  # Store first 100 characters as a preview of the chunk
#                 }
#                 # Append vector with ID and metadata
#                 vectors.append({"id": chunk_id, "values": vector, "metadata": metadata})

#             # Upsert the vectors (chunks) into Pinecone with metadata
#             index.upsert(vectors=vectors, namespace="")

#             #print(f"Uploaded Document - Prefix: {doc_id_prefix}, Total Chunks: {len(chunks)}")
#             print(f"Uploaded Document - Filename: {filename}, embedded_ID: {doc_id_prefix}, Total Chunks: {len(chunks)}")

#     print("All documents uploaded successfully!")

# # Function to list all chunks of a document by prefix
# def list_document_chunks(doc_id_prefix):
#     chunk_ids = []
#     for ids in index.list(prefix=f"{doc_id_prefix}#", namespace=""):
#         chunk_ids.extend(ids)
#     return chunk_ids

# # Example call to upload documents from the directory
# tenant_ID = "tenant1234"  # Example tenant ID
# upload_documents(directory, embeddings, index, tenant_ID)


In [50]:
# import os
# import random
# from PyPDF2 import PdfReader  # For reading PDF files
# import docx
# from langchain.document_loaders import DirectoryLoader
# from langchain.text_splitter import RecursiveCharacterTextSplitter
# from concurrent.futures import ThreadPoolExecutor, as_completed

# # Define constants
# chunk_size = 1000  # Adjust based on your use case
# chunk_overlap = 100  # Adjust based on your use case

# # Function to read text from different file types
# def read_document(file_path):
#     if file_path.endswith('.txt'):
#         with open(file_path, 'r', encoding='utf-8') as file:
#             return file.read()
#     elif file_path.endswith('.pdf'):
#         with open(file_path, 'rb') as file:
#             reader = PdfReader(file)
#             return "\n".join(page.extract_text() for page in reader.pages if page.extract_text())
#     elif file_path.endswith('.docx'):
#         doc = docx.Document(file_path)
#         return "\n".join(paragraph.text for paragraph in doc.paragraphs)
#     else:
#         return None  # Unsupported file type

# # Function to generate a 14-digit random ID
# def generate_14_digit_random_id():
#     return str(random.randint(10000000000000, 99999999999999))

# # Function to process each document
# def process_document(filename, directory, embeddings, index, tenant_ID):
#     file_path = os.path.join(directory, filename)
#     document_text = read_document(file_path)
    
#     if document_text:
#         # Generate a random 14-digit document ID prefix
#         doc_id_prefix = generate_14_digit_random_id()

#         # Split the document into chunks
#         text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
#         chunks = text_splitter.split_text(document_text)

#         # Generate embeddings for chunks in batches
#         vectors = []
#         for i, chunk in enumerate(chunks):
#             chunk_id = f"{doc_id_prefix}#chunk{i+1}"
#             vectors.append({"id": chunk_id, "content": chunk})

#         # Generate embeddings for all chunks at once
#         if vectors:
#             content_list = [vector["content"] for vector in vectors]
#             embedded_vectors = embeddings.embed_documents(content_list)

#             # Prepare the vectors for upsert
#             for vector, embedding in zip(vectors, embedded_vectors):
#                 vector["values"] = embedding
#                 vector["metadata"] = {
#                     "source": filename,
#                     "tenant_ID": tenant_ID,
#                     "page_content": vector["content"][:100],  # Store first 100 characters as a preview
#                 }

#             # Upsert the vectors (chunks) into Pinecone with metadata
#             index.upsert(vectors=[{"id": vector["id"], "values": vector["values"], "metadata": vector["metadata"]} for vector in vectors], namespace="")
#             print(f"Uploaded Document - Filename: {filename}, embedded_ID: {doc_id_prefix}, Total Chunks: {len(chunks)}")

# # Main upload function using threading
# def upload_documents(directory, embeddings, index, tenant_ID):
#     with ThreadPoolExecutor() as executor:
#         futures = []
#         for filename in os.listdir(directory):
#             if filename.endswith(('.txt', '.pdf', '.docx')):  # Process only supported files
#                 futures.append(executor.submit(process_document, filename, directory, embeddings, index, tenant_ID))

#         # Wait for all futures to complete
#         for future in as_completed(futures):
#             future.result()  # This will also raise exceptions if any occurred during processing

#     print("All documents uploaded successfully!")

# # Example call to upload documents from the directory
# tenant_ID = "tenant1234"  # Example tenant ID
# directory = 'data'  # Specify the directory containing the documents
# upload_documents(directory, embeddings, index, tenant_ID)


In [51]:
# import os
# import random
# from PyPDF2 import PdfReader  # For reading PDF files
# import docx
# from langchain.document_loaders import DirectoryLoader
# from langchain.text_splitter import RecursiveCharacterTextSplitter
# from concurrent.futures import ThreadPoolExecutor, as_completed

# # Define constants
# chunk_size = 1000  # Adjust based on your use case
# chunk_overlap = 100  # Adjust based on your use case

# # Function to read text from different file types
# def read_document(file_path):
#     if file_path.endswith('.txt'):
#         with open(file_path, 'r', encoding='utf-8') as file:
#             return file.read()
#     elif file_path.endswith('.pdf'):
#         with open(file_path, 'rb') as file:
#             reader = PdfReader(file)
#             return "\n".join(page.extract_text() for page in reader.pages if page.extract_text())
#     elif file_path.endswith('.docx'):
#         doc = docx.Document(file_path)
#         return "\n".join(paragraph.text for paragraph in doc.paragraphs)
#     else:
#         return None  # Unsupported file type

# # Function to generate a 14-digit random ID
# def generate_14_digit_random_id():
#     return str(random.randint(10000000000000, 99999999999999))

# # Function to process each document
# def process_document(filename, directory, embeddings, index):
#     file_path = os.path.join(directory, filename)
#     document_text = read_document(file_path)
    
#     if document_text:
#         # Generate a random 14-digit document ID prefix
#         doc_id_prefix = generate_14_digit_random_id()

#         # Split the document into chunks
#         text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
#         chunks = text_splitter.split_text(document_text)

#         # Generate embeddings for chunks in batches
#         vectors = []
#         for i, chunk in enumerate(chunks):
#             chunk_id = f"{doc_id_prefix}#chunk{i+1}"
#             vectors.append({"id": chunk_id, "content": chunk})

#         # Generate embeddings for all chunks at once
#         if vectors:
#             content_list = [vector["content"] for vector in vectors]
#             embedded_vectors = embeddings.embed_documents(content_list)

#             # Prepare the vectors for upsert
#             for vector, embedding in zip(vectors, embedded_vectors):
#                 vector["values"] = embedding
#                 vector["metadata"] = {
#                     "source": filename,
#                     "page_content": vector["content"][:100],  # Store first 100 characters as a preview
#                 }

#             # Upsert the vectors (chunks) into Pinecone with metadata
#             index.upsert(vectors=[{"id": vector["id"], "values": vector["values"], "metadata": vector["metadata"]} for vector in vectors], namespace="")
#             print(f"Uploaded Document - Filename: {filename}, embedded_ID: {doc_id_prefix}, Total Chunks: {len(chunks)}")

# # Main upload function using threading
# def upload_documents(directory, embeddings, index):
#     with ThreadPoolExecutor() as executor:
#         futures = []
#         for filename in os.listdir(directory):
#             if filename.endswith(('.txt', '.pdf', '.docx')):  # Process only supported files
#                 futures.append(executor.submit(process_document, filename, directory, embeddings, index))

#         # Wait for all futures to complete
#         for future in as_completed(futures):
#             future.result()  # This will also raise exceptions if any occurred during processing

#     print("All documents uploaded successfully!")

# # Example call to upload documents from the directory
# directory = 'data'  # Specify the directory containing the documents
# upload_documents(directory, embeddings, index)


In [76]:
#chunk_size = 500 
#chunk_overlap = 50  

text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)

split_docs = text_splitter.split_documents(docs)

In [77]:
index_name = "test-2"
#vectorstore = PineconeVectorStore(index_name=index_name, embedding=embeddings)
vectorstore = PineconeVectorStore.from_documents(split_docs, embeddings, index_name=index_name)
retriever = vectorstore.as_retriever()

In [78]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

retrieved_docs = retriever.invoke("""what is The Myth of Prometheus in Literary Texts """)
print(format_docs(retrieved_docs))

Found document with no `text` key. Skipping.
Found document with no `text` key. Skipping.
Found document with no `text` key. Skipping.


To edit template content click on the default templates from the list, type Subject then type email content using the metadata buttons on the bottom of the page, click Save or to publish click Publish/Unpublish button on the top right.

NOTE: To make sure clients are receiving email check the publication status on the top left corner beside the template name shown as gray box.

To delete a template click on the trash icon in the top right of the template box.


In [79]:
llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0.0,
)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(),

)
qa.invoke("how to creat a client")

Found document with no `text` key. Skipping.
Found document with no `text` key. Skipping.
Found document with no `text` key. Skipping.


{'query': 'how to creat a client',
 'result': 'To create an individual client, follow these steps:\n\n1. Go to the clients module from the left menu bar and select the "individual" tab.\n2. Click the "+Add" button.\n3. Complete the online form that appears. You will need to select one of the four client types: Primary, Dependent, Group, or Group Member.\n4. If you are adding a Primary client, fill in the required information such as Name, Date of Birth, Email, Phone, and Address.\n5. Click "Save" to save the client\'s information.\n6. Next, select the desired offering and add the start date.'}

In [56]:
# custom_rag_template = PromptTemplate.from_template(template)

# # Create the parallel chain
# My_rag_chain = RunnableParallel(
#     {
#         "context": retriever | format_docs,
#         "question": RunnablePassthrough()
#     }
# ) | custom_rag_template | llm | StrOutputParser()

# ## My chain : Retriever(Pinecone) | custom_rag_template(prompt) | llm | StrOutputParser()

In [80]:
keyword = "How to create a client?"
template = """You are an expert LLM assistant specialized in answering questions based solely on the information provided in the uploaded documents (PDF, DOCX, or TXT formats). Use only the information from the documents to respond accurately and clearly to each question.

Guidelines:
1. Avoid using outside knowledge or assumptions. Stick strictly to the content in the documents.
2. If the answer is not found in the uploaded documents, state, "The answer is not specifically mentioned in the provided documents."
3. Avoid using outside knowledge or assumptions. Stick strictly to the content in the documents.
4. Maintain a professional and helpful tone thinking you are giving service to the customer for their documents.
5. Answer for normal conversation questions like "Hi", "Hey", "Hello", "How are you?", and many others with the answer: "Hello, How can I assist you?".
6. If the question is on "summarize" or "summarization", then summarize the documents to (1/4)th the size of the original documents.

Question: {question}

Context: {context}

Answer:
"""

prompt = template.format(question=keyword, context=format_docs(retrieved_docs))

# Create the LLM
llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0.1,
)

# Set up the retrieval QA with custom response for missing answers
qa = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectorstore.as_retriever(),
    return_source_documents=True  # Ensures that source documents are returned
)

# Wrapper function to handle missing answers
def get_answer(keyword):
    response = qa.invoke(keyword)
    
    # Check if the response contains valid data
    result = response.get('result', '')
    if "The answer is not specifically mentioned" in result or not result.strip():
        return "Precise answer not found in documents, try another prompt."
    else:
        return result

# Test the response with the keyword quer
final_answer = get_answer("how to create a client")
print(final_answer)

Found document with no `text` key. Skipping.
Found document with no `text` key. Skipping.
Found document with no `text` key. Skipping.


To create an individual client, follow these steps:

1. Go to the clients module from the left menu bar and select the "individual" tab.
2. Click the "+Add" button.
3. You will be presented with an online form to complete. There are four different client types: Primary, Dependent, Group, and Group Member.
4. Select "Primary" and fill in the required information such as Name, Date of Birth, Email, Phone, and Address.
5. Click "Save."
6. Next, select the desired offering and add the start date.


In [81]:
pc.describe_index("test-2")


{
    "name": "test-2",
    "dimension": 1536,
    "metric": "dotproduct",
    "host": "test-2-5vwf04k.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "deletion_protection": "disabled"
}

In [59]:
qa.invoke("how to modify a membership")

Found document with no `text` key. Skipping.
Found document with no `text` key. Skipping.


{'query': 'how to modify a membership',
 'result': "To modify a membership, follow these steps:\n\n1. **Go to the clients module** from the left menu bar.\n2. **Find the client** you want to update membership for.\n3. **Click on the ellipses** (three dots) next to the client's name and select **modify membership**.\n4. In the **modify membership** popup, you can view all currently active memberships as well as past inactive memberships.\n5. To **change or update a membership**, specify a new start date for the desired benefit.\n6. If you want to **add an additional membership**, click **+Add** on the modify membership popup, select the membership from the list, choose between Public/Private bundle (Private can only be chosen/seen by admin), add the start date of the new membership, and then select **Continue** and **Confirm**.\n7. To **cancel an existing membership**, you can also do this from the modify membership section by selecting the option to cancel and add an end date.\n\nNote:

In [60]:
# # Function to delete all chunks of a document by prefix
# def delete_document_by_prefix(doc_id_prefix):
#     chunk_ids = list_document_chunks(doc_id_prefix)
#     if chunk_ids:
#         index.delete(ids=chunk_ids, namespace="")
#         print(f"Deleted {len(chunk_ids)} chunks for document: {doc_id_prefix}")
#     else:
#         print(f"No chunks found for document: {doc_id_prefix}")

# # Example usage
# # Upload documents from the directory
# upload_documents(directory, embeddings, index)

# # List all chunk IDs for a document
# doc_id_prefix = "cricket_rules"  # Replace with your actual document name
# chunks = list_document_chunks(doc_id_prefix)
# print(f"Chunks for document '{doc_id_prefix}': {chunks}")

# # Delete all chunks for a document
# delete_document_by_prefix(doc_id_prefix)

In [82]:
#index.delete(delete_all=True)


{}